# Convolutional Layer

In [1]:
import numpy as np, sys
np.random.seed(1)
from keras.datasets import mnist

# Load MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Prepare training data: take first 1000 images, flatten and normalize
images, labels = (x_train[0:1000].reshape(1000,28*28) / 255, y_train[0:1000])

# Convert labels to one-hot encoding
one_hot_labels = np.zeros((len(labels),10))
for i,l in enumerate(labels):
    one_hot_labels[i][l] = 1
labels = one_hot_labels

# Prepare test data: flatten and normalize images, convert labels to one-hot
test_images = x_test.reshape(len(x_test),28*28) / 255
test_labels = np.zeros((len(y_test),10))
for i,l in enumerate(y_test):
    test_labels[i][l] = 1

# Activation function: hyperbolic tangent
def tanh(x):
    return np.tanh(x)

# Derivative of tanh for backpropagation
def tanh2deriv(output):
    return 1 - (output ** 2)

# Softmax activation function for output layer
def softmax(x):
    temp = np.exp(x)
    return temp / np.sum(temp, axis=1, keepdims=True)

# Hyperparameters
alpha, iterations = (2, 300)  # Learning rate and number of training iterations
pixels_per_image, num_labels = (784, 10)  # Input size and number of classes
batch_size = 128  # Number of samples per batch

# Input image dimensions
input_rows = 28
input_cols = 28

# Convolutional kernel dimensions
kernel_rows = 3
kernel_cols = 3
num_kernels = 16

# Calculate size of hidden layer after convolution
hidden_size = ((input_rows - kernel_rows) * (input_cols - kernel_cols)) * num_kernels

# Initialize weights
kernels = 0.02*np.random.random((kernel_rows*kernel_cols, num_kernels))-0.01
weights_1_2 = 0.2*np.random.random((hidden_size, num_labels)) - 0.1

# Function to extract a section of the image for convolution
def get_image_section(layer, row_from, row_to, col_from, col_to):
    section = layer[:,row_from:row_to,col_from:col_to]
    return section.reshape(-1,1,row_to-row_from, col_to-col_from)

# Training loop
for j in range(iterations):
    correct_cnt = 0
    # Iterate over batches
    for i in range(int(len(images) / batch_size)):
        batch_start, batch_end = ((i * batch_size), ((i+1) * batch_size))
        
        # Prepare input layer
        layer_0 = images[batch_start:batch_end]
        layer_0 = layer_0.reshape(layer_0.shape[0], 28, 28)
        
        # Apply convolution
        sects = list()
        for row_start in range(layer_0.shape[1] - kernel_rows):
            for col_start in range(layer_0.shape[2] - kernel_cols):
                sect = get_image_section(layer_0,
                                         row_start, row_start+kernel_rows,
                                         col_start, col_start+kernel_cols)
                sects.append(sect)
        
        expanded_input = np.concatenate(sects, axis=1)
        es = expanded_input.shape
        flattened_input = expanded_input.reshape(es[0]*es[1], -1)
        
        # Forward pass
        kernel_output = flattened_input.dot(kernels)
        layer_1 = tanh(kernel_output.reshape(es[0], -1))
        
        # Apply dropout
        dropout_mask = np.random.randint(2, size=layer_1.shape)
        layer_1 *= dropout_mask * 2
        
        # Output layer
        layer_2 = softmax(np.dot(layer_1, weights_1_2))
        
        # Calculate accuracy for this batch
        for k in range(batch_size):
            labelset = labels[batch_start+k:batch_start+k+1]
            correct_cnt += int(np.argmax(layer_2[k:k+1]) == np.argmax(labelset))
        
        # Backpropagation
        layer_2_delta = (labels[batch_start:batch_end] - layer_2) / (batch_size * layer_2.shape[0])
        layer_1_delta = layer_2_delta.dot(weights_1_2.T) * tanh2deriv(layer_1)
        layer_1_delta *= dropout_mask
        
        # Update weights
        weights_1_2 += alpha * layer_1.T.dot(layer_2_delta)
        l1d_reshape = layer_1_delta.reshape(kernel_output.shape)
        k_update = flattened_input.T.dot(l1d_reshape)
        kernels -= alpha * k_update

    # Testing loop
    test_correct_cnt = 0
    for i in range(len(test_images)):
        # Prepare input layer
        layer_0 = test_images[i:i+1]
        layer_0 = layer_0.reshape(layer_0.shape[0], 28, 28)
        
        # Apply convolution
        sects = list()
        for row_start in range(layer_0.shape[1] - kernel_rows):
            for col_start in range(layer_0.shape[2] - kernel_cols):
                sect = get_image_section(layer_0,
                                         row_start, row_start+kernel_rows,
                                         col_start, col_start+kernel_cols)
                sects.append(sect)
        
        expanded_input = np.concatenate(sects, axis=1)
        es = expanded_input.shape
        flattened_input = expanded_input.reshape(es[0]*es[1], -1)
        
        # Forward pass (without dropout for testing)
        kernel_output = flattened_input.dot(kernels)
        layer_1 = tanh(kernel_output.reshape(es[0], -1))
        layer_2 = np.dot(layer_1, weights_1_2)
        
        # Calculate test accuracy
        test_correct_cnt += int(np.argmax(layer_2) == np.argmax(test_labels[i:i+1]))

    # Print progress every iteration
    if (j % 1 == 0):
        sys.stdout.write("\n" + 
                         "I:" + str(j) + 
                         " Test-Acc:" + str(test_correct_cnt/float(len(test_images))) +
                         " Train-Acc:" + str(correct_cnt/float(len(images))))

ModuleNotFoundError: No module named 'tensorflow'